# Day 1

In [3]:
turns = [int(i.replace("L", "-").replace("R", "")) for i in open("data/day1").read().strip().split("\n")]

sum([1 if (50+sum(turns[:i]))%100==0 else 0 for i in range(1, len(turns))])

1097

In [4]:
val = 50
counter = 0

for i in turns:
    if abs(i)//100 != 0:
        counter += abs(i)//100
        if i < 0:
            i -= (abs(i)//100)*-100
        else:
            i %= 100
    val += i
    if (abs(val) != abs(i)) and (val < 0 or val > 100):
        counter+= 1
    val %= 100
    if val == 0:
        counter += 1

counter

7101

# Day 2

In [1]:
%%timeit 

ranges = [list(map(int, i.split("-"))) for i in open("data/day2").read().replace("\n", "").split(",")]

invalid = 0
for rng in ranges:
    for i in range(rng[0], rng[1]+1):
        strint = str(i)
        if len(strint)%2 == 0:
            if strint[:len(strint)//2] == strint[len(strint)//2:]:
                invalid += int(strint)

invalid

213 ms ± 5.15 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [3]:
ranges = [list(map(int, i.split("-"))) for i in open("data/day2").read().replace("\n", "").split(",")]

invalid = 0
for rng in ranges:
    for i in range(rng[0], rng[1]+1):
        strint = str(i)
        for j in range(1, len(strint)//2+1):
            result = set(strint.split(strint[:j]))
            if len(result) == 1:
                invalid += i
                break
invalid

1.22 s ± 12.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [1]:
# second alternative to part 2, because regex is very powerful


# Day 3

In [2]:
input = open("data/day3").read().strip().split("\n")

total = 0
for i in input:
    digitmax = 0
    digits = list(map(int, i))
    for j, val in enumerate(digits):
        for l, val2 in enumerate(digits):
            if j != l and j < l:
                current = val * 10 + val2
                if current > digitmax:
                    digitmax = current
    total += digitmax
total

16858

In [3]:
input = open("data/test").read().strip().split("\n")

import re

total = 0
for i in input:
    currentmax = 0
    digits_remaining = 12
    highest_digit = 9
    while digits_remaining > 0:
        item = str(highest_digit) + "\\d" + "{" + str(digits_remaining-1) + "}"
        blah = re.search(item, i)
        if blah:
            currentmax += highest_digit * (10 ** (digits_remaining-1))
            i = i[i.index(str(highest_digit))+1:]
            digits_remaining -= 1
            highest_digit = 9
        else:
            highest_digit -= 1
    total += currentmax

total

3121910778619

# Day 4

In [1]:
positions = open("data/day4").read().strip().split("\n")

def perimeter(y, x, positions):
    if positions[y][x] != '@':
        return False
    toilet_rolls = 0
    left = max(0, x-1)
    right = min(x+1, len(positions[0])-1)
    up = max(0, y-1)
    down = min(y+1, len(positions)-1)

    for i in range(up, down+1):
        for j in range(left, right+1):
            if positions[i][j] == "@":
                toilet_rolls += 1
    toilet_rolls-= 1

    if toilet_rolls < 4:
        return True
    else:
        return False

counter = 0
for i in range(len(positions)):
    for j in range(len(positions[0])):
        if perimeter(i, j, positions):
            counter += 1

counter

1419

In [2]:
positions = open("data/day4").read().strip().split("\n")

def perimeter(y, x, positions):
    if positions[y][x] != '@':
        return False
    toilet_rolls = 0
    left = max(0, x-1)
    right = min(x+1, len(positions[0])-1)
    up = max(0, y-1)
    down = min(y+1, len(positions)-1)

    for i in range(up, down+1):
        for j in range(left, right+1):
            if positions[i][j] == "@":
                toilet_rolls += 1
    toilet_rolls-= 1

    if toilet_rolls < 4:
        return True
    else:
        return False


def clean(positions, counter):
    for i in range(len(positions)):
        for j in range(len(positions[0])):
            if perimeter(i, j, positions):
                counter += 1
                positions[i] = positions[i][:j] + '.' + positions[i][j+1:]
    return positions, counter

counter = 0
while True:
    positions, new_counter = clean(positions, counter)
    if new_counter == counter:
        break
    else:
        counter = new_counter 
counter

8739

# Day 4 Alternate (Numpy Edition)

In [11]:
import numpy as np

tp = np.array([list(map(lambda x: int(x),i)) for i in open("data/day4").read().replace("@", "1").replace(".","0").strip().split("\n")])

(sum([1 if 
      (np.sum(tp[max(x-1,0):min(tp.shape[0], x+2), max(y-1,0):min(tp.shape[0], y+2)])-1 if tp[x,y] else 10) < 4 
      else 0 
      for x, y in np.ndindex(tp.shape)])
)

1419

In [12]:
import numpy as np

tp = np.array([list(map(lambda x: int(x),i)) for i in open("data/day4").read().replace("@", "1").replace(".","0").strip().split("\n")])
convolve = lambda x, y, mt: np.sum(mt[max(x-1,0):min(mt.shape[0], x+2), max(y-1,0):min(mt.shape[0], y+2)])-1 if mt[x,y] else 10

def check(mt):
    if (total:=sum([1 if convolve(x, y, mt) < 4 else 0 for x, y in np.ndindex(mt.shape)])) != 0:
        return total + check(np.array([0 if convolve(x, y, mt) < 4 else mt[x,y] 
                                       for x, y in np.ndindex(mt.shape)]).reshape(mt.shape))
    else:
        return 0

check(tp)

8739

# Day 5

In [4]:
ranges, ingredients = [i.split("\n") for i in open("data/day5").read().strip().split("\n\n")]

actual_ranges = []
for r in ranges:
    beg, end = r.split("-")
    actual_ranges.append(range(int(beg), int(end)+1))

fresh = 0
for i in ingredients:
    for r in actual_ranges:
        if int(i) in r:
            fresh += 1
            break

fresh

638

In [5]:
ranges, ingredients = [i.split("\n") for i in open("data/day5").read().strip().split("\n\n")]

actual_ranges = []
for r in ranges:
    beg, end = r.split("-")
    actual_ranges.append([int(beg), int(end)])

def clean_ranges(actual_ranges):
    final_ranges = []
    for r in actual_ranges:
        added = False
        for idx, fr in enumerate(final_ranges):
            if fr[0] <= r[0] <= fr[1]:
                if r[1] < fr[0] or r[1] > fr[1]:
                    final_ranges[idx] = [fr[0], r[1]]
                    added = True
                    break
                elif fr[0] <= r[1] <= fr[1]:
                    added = True
                    break

            elif fr[0] <= r[1] <= fr[1]:
                if r[0] < fr[0] or r[0] > fr[1]:
                    final_ranges[idx] = [r[0], fr[1]]
                    added = True
                    break
                elif fr[0] <= r[0] <= fr[1]:
                    added = True
                    break

        if not added:
            final_ranges.append(r)
            added = False

    final_ranges.sort(key=lambda x: x[0])
    return final_ranges

prev_length = 0
current_length = 1
while current_length != prev_length:
    prev_length = current_length
    actual_ranges = clean_ranges(actual_ranges)
    current_length = len(actual_ranges)

total = 0
for item in actual_ranges:
    total += item[1]-item[0]+1

total

352946349407338

# Day 6

In [1]:
import math
results = list(zip(*[i.split() for i in open("data/day6").read().strip().split("\n")]))

sum([(math.prod if results[i][-1] == "*" else sum)(map(int, results[i][:-1])) for i in range(len(results))])


6343365546996

In [3]:
results, ops =  [i.split(",") for i in ",".join(("".join(i) for i in zip(*[i for i in open("data/day6").read().strip().split("\n")][:-1]))).split(f",    ,")], open("data/day6").read().strip().split("\n")[-1].split()

sum([(math.prod if ops[i] == "*" else sum)(map(int, results[i])) for i in range(len(results))])

11136895955912

# Day 7

In [6]:
space = open("data/day7").read().strip().split("\n")

pews = set([(0, space[0].index("S"))])
splitters = []
for x in range(len(space)):
    for y, val in enumerate(space[x]):
        if val == "^":
            splitters.append((x, y))
    
splits = 0
    
def getpew(pews):
    global splits
    new_pews = set()
    for pew in pews:
        if pew[0] > len(space):
            return False, set()
        if (pew[0]+1, pew[1]) in splitters:
            splits += 1
            new_pews.add((pew[0]+1, pew[1]-1))
            new_pews.add((pew[0]+1, pew[1]+1))
        else:
            new_pews.add((pew[0]+1, pew[1]))
    return True, new_pews

while True:
    cont, pews = getpew(pews)
    if not cont:
        break

splits

1698

In [7]:
from functools import lru_cache 
space = open("data/day7").read().strip().split("\n")

start = (0, space[0].index("S"))

@lru_cache
def make_path(x, y):
    if x+1 >= len(space):
        return 1
    if space[x+1][y] == "^":
        return make_path(x+1, y-1) + make_path(x+1, y+1)
    else:
        return make_path(x+1, y)

make_path(start[0], start[1])

95408386769474

# Day 8

In [9]:
boxes = [tuple(map(int, i.split(","))) for i in open("data/day8").read().strip().split("\n")]

topK = 1000
distances = {}
for boxA in boxes:
    for boxB in boxes:
        if boxA != boxB and frozenset([boxA, boxB]) not in distances:
            result = sum((a - b)**2 for a, b in zip(boxA, boxB))**.5
            distances[frozenset([boxA, boxB])] = result

circuits = []
distances = [i[0] for i in (sorted(distances.items(), key=lambda x: x[1]))][:topK]

collapse = True
ldistance = len(distances)
while collapse:
    for idx in range(len(distances)):
        for idx2 in range(len(distances)):
            if distances[idx] is not None and distances[idx2] is not None and idx!=idx2:
                if distances[idx].intersection(distances[idx2]):
                    distances[idx] = distances[idx].union(distances[idx2])
                    distances[idx2] = None
    distances = [i for i in distances if i is not None]
    if ldistance == len(distances):
        collapse = False
    ldistance = len(distances)
    
results = sorted(distances, key=lambda x: -len(x))
len(results[0]) * len(results[1]) * len(results[2])

52668

In [10]:
boxes = [tuple(map(int, i.split(","))) for i in open("data/day8").read().strip().split("\n")]

topK = 1000
distances = {}
for boxA in boxes:
    for boxB in boxes:
        if boxA != boxB and frozenset([boxA, boxB]) not in distances:
            result = sum((a - b)**2 for a, b in zip(boxA, boxB))**.5
            distances[frozenset([boxA, boxB])] = result

circuits = []
orig_distances = [i[0] for i in (sorted(distances.items(), key=lambda x: x[1]))]
distances = [i[0] for i in (sorted(distances.items(), key=lambda x: x[1]))][:topK]
last = list(distances[-1])[0][0] * list(distances[-1])[1][0]


while True: 
    collapse = True
    ldistance = len(distances)
    while collapse:
        for idx in range(len(distances)):
            for idx2 in range(len(distances)):
                if distances[idx] is not None and distances[idx2] is not None and idx!=idx2:
                    if distances[idx].intersection(distances[idx2]):
                        distances[idx] = distances[idx].union(distances[idx2])
                        distances[idx2] = None
        distances = [i for i in distances if i is not None]
        if ldistance == len(distances):
            collapse = False
        ldistance = len(distances)

    if len(distances) != 1 or len(distances[0]) != len(boxes):
        last = list(orig_distances[topK])[0][0] * list(orig_distances[topK])[1][0]
        distances.append(orig_distances[topK])
        topK+=1
    else:
        break

last

1474050600

# Day 9

In [14]:
import math
coords = [tuple(map(int, i.split(","))) for i in open("data/day9").read().strip().split("\n")]

areas = {}
for cornerA in coords:
    for cornerB in coords:
        if cornerA != cornerB and (corners:=frozenset([cornerA, cornerB])) not in areas:
            areas[corners] = math.prod((abs(x-y)+1 for x, y in zip(cornerA, cornerB)))
max(areas.values())

4746238001

In [15]:
import math 

coords = [tuple(map(int, i.split(","))) for i in open("data/day9").read().strip().split("\n")]
mx, my = map(lambda x: max(x)+1, zip(*coords))
pm = {}
edges = list(zip(coords, coords[1:]+[coords[0]]))

areas = {}
for cornerA in coords:
    for cornerB in coords:
        if cornerA != cornerB and (corners:=frozenset([cornerA, cornerB])) not in areas:
            areas[corners] = math.prod((abs(x-y)+1 for x, y in zip(cornerA, cornerB)))

areas = sorted(areas.items(), key=lambda x: -x[1])

for idx, (x1, x2) in enumerate(edges):
    goingUp = [False, False]
    if x1[1] == x2[1]:
        # first get direction of the horizontals
        pre = edges[idx-1][0]
        if pre[1] > x1[1]:
            goingUp[0] = True

        post = edges[(idx+1)%len(edges)][1]
        if x1[1] > post[1]:
            goingUp[1] = True

        if goingUp[0] == goingUp[1]:
            txt = "change"
        else:
            txt = "complete"
            
        # now swap the order as needed
        if x1[0] > x2[0]:
            x1, x2 = x2, x1

        if x1[1] in pm:
            pm[x1[1]].append((x1[0], x2[0]+1, txt))
        else:
            pm[x1[1]] = [(x1[0], x2[0]+1, txt)]

for idx, (x1, x2) in enumerate(edges):
    if x1[0] == x2[0]:
        x = x1[0]
        y1, y2 = x1[1], x2[1]
        if y1 > y2:
            y1, y2 = y2, y1
        for y in range(y1, y2+1):
            # print(x1, x2, y)

            found = False
            if y in pm:
                ranges = pm[y]

                for r in ranges:
                    if x in range(r[0], r[1]):
                        found = True
                    
            if not found:                    
                if y in pm:
                    pm[y].append((x, x+1, "change"))
                else:
                    pm[y] = [(x, x+1, "change")]

for k, v in pm.items():
    v = sorted(v, key=lambda x: x[0])
    v.insert(0, (0, v[0][0], "out"))

    if v[-1][1] != mx:
        v.insert(-1, (v[-1][1], mx, "unk"))
    v = sorted(v, key=lambda x: x[0])

    for a, b in zip(v,v[1:]):
        if a[1] != b[0]:
            r = (a[1], b[0], "unk")
            v.append(r)            
    v = sorted(v, key=lambda x: x[0])

    out = True

    for idx in range(len(v)):
        if v[idx][-1] == 'out':
            out == True
        if v[idx][-1] == 'unk':
            txt = 'out' if out else 'in'
            v[idx] = (v[idx][0], v[idx][1], txt)
        if v[idx][-1] == 'change':
            out = not out
    pm[k] = v


def testIn(cA, cB):
    x1, y1 = cA
    x2, y2 = cB
    if x1 > x2:
        x1, x2 = x2, x1
    if y1 > y2:
        y1, y2 = y2, y1

    for y in range(y1, y2+1):
        if y not in pm:
            return False

        start, end = 0, 0
        for idx in range(len(pm[y])):
            if x1 in range(pm[y][idx][0], pm[y][idx][1]):
                start = idx
            if x2 in range(pm[y][idx][0], pm[y][idx][1]):
                end = idx
        for i in pm[y][start:end+1]:
            if i[-1] == "out":
                return False
    
    return True
    
for corners, area in areas:
    cA, cB = corners
    if testIn(cA, cB):
        print(area)
        break

1552139370


# Day 10

In [16]:
from itertools import combinations_with_replacement
from functools import reduce
items = [i.split("]") for i in open("data/day10").read().strip().split("\n")]

lights = ""
switches = []

factory = []
for i in items:
    lights = int(i[0][1:].replace(".", "0").replace("#", "1"),2)
    switches = [eval(j.replace(")", ",)")) for j in i[1].strip().split("{")[0].strip().split()]
    switches = [int("".join(["1" if pos in switch else "0" for pos in range(len(i[0][1:]))]),2) for switch in switches]
    factory.append((lights, switches))

def presses(light, switches):
    press = 1
    while True:
        check = combinations_with_replacement(switches, press)
        for c in check:
            if reduce(lambda x, y: x^y, c) == light:
                return press
        press += 1
        

sum([presses(l, s) for l, s in factory])

449

In [17]:
from ortools.linear_solver import pywraplp

items = [i.split("]") for i in open("data/day10").read().strip().split("\n")]

lights = ""
switches = []

factory = []
for i in items:
    lights = int(i[0][1:].replace(".", "0").replace("#", "1"),2)
    switches = [eval(j.replace(")", ",)")) for j in i[1].strip().split("{")[0].strip().split()]
    switches = [[1 if pos in switch else 0 for pos in range(len(i[0][1:]))] for switch in switches]
    jolts = list(map(int, i[1].strip().split("{")[1].replace("}", "").split(",")))
    factory.append((jolts, switches))


def minimize(target, switches):
    solver = pywraplp.Solver.CreateSolver("CBC")

    x = [solver.IntVar(0, solver.infinity(), f'x{i}') for i in range(len(switches))]
    
    for j in range(len(target)): 
        solver.Add(sum(x[i] * switches[i][j] for i in range(len(switches))) == target[j])
    
    solver.Minimize(sum(x))
    
    status = solver.Solve()
    
    return  solver.Objective().Value()
            
int(sum([minimize(target, switches) for target, switches in factory]))

17848

# Day 11

In [18]:
path = {i.split(":")[0]:i.split(":")[1].strip().split(" ") for i in open("data/day11").read().strip().split("\n")}

def getout(current):
    if "out" in path[current]:
        return 1
    else:
        return sum([getout(v) for v in path[current]])

getout("you")

571

In [19]:
from functools import lru_cache

path = {i.split(":")[0]:i.split(":")[1].strip().split(" ") for i in open("data/day11").read().strip().split("\n")}
    
@lru_cache
def getout(current, found_dac, found_fft):
    if current == 'dac':
        found_dac = True
    if current == 'fft':
        found_fft = True
    if "out" in path[current] and found_dac and found_fft:
        return 1
    elif "out" in path[current]:
        return 0
    else:
        return sum([getout(v, found_dac, found_fft) for v in path[current]])

getout("svr", False, False)

511378159390560